In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats as st

In [60]:
df = pd.read_csv('/Users/vitikhonov/Desktop/marketing_AB.csv')
df

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14
...,...,...,...,...,...,...,...
588096,588096,1278437,ad,False,1,Tuesday,23
588097,588097,1327975,ad,False,1,Tuesday,23
588098,588098,1038442,ad,False,3,Tuesday,23
588099,588099,1496395,ad,False,1,Tuesday,23


In [61]:
df['user id'].value_counts()

1069124    1
1116707    1
1275145    1
1338348    1
1352590    1
          ..
1381349    1
1526572    1
1144916    1
1321771    1
1237779    1
Name: user id, Length: 588101, dtype: int64

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [63]:
df.describe()

,Unnamed: 0,user id,total ads,most ads hour
count,588101.000000,5.881010e+05,588101.000000,588101.000000
mean,294050.000000,1.310692e+06,24.820876,14.469061
std,169770.279668,2.022260e+05,43.715181,4.834634
min,0.000000,9.000000e+05,1.000000,0.000000
25%,147025.000000,1.143190e+06,4.000000,11.000000
50%,294050.000000,1.313725e+06,13.000000,14.000000
75%,441075.000000,1.484088e+06,27.000000,18.000000
max,588100.000000,1.654483e+06,2065.000000,23.000000


Для анализа нам представлен датасет marketing_AB, который содержит следующую информацию:

- user id: Идентификатор пользователя (уникальный).
- test group: Если "ad", то человек видел рекламу, если "psa", то он видел только объявление государственной службы.
- converted: Если человек купил продукт, то True, иначе False.
- total ads: Количество рекламы, увиденной человеком.
- most ads day: День, в который человек увидел наибольшее количество рекламы.
- most ads hour: Час дня, в который человек увидел наибольшее количество рекламы.

При первичном анализе видно, что  датасет состоит из 588101 строки и 7 колонок. Все данные ненулевые.
User ID не повторяется, то есть находится только в одной группе.

### Подготовка данных

Так как user id уникален, то по сути он дублирует индекс и номер ячейки - необходимо установить его в качестве индекса.

In [64]:
df = df.drop(['Unnamed: 0'], axis=1)
df = df.set_index('user id')
df

,test group,converted,total ads,most ads day,most ads hour
user id,,,,,
1069124,ad,False,130,Monday,20
1119715,ad,False,93,Tuesday,22
1144181,ad,False,21,Tuesday,18
1435133,ad,False,355,Tuesday,10
1015700,ad,False,276,Friday,14
...,...,...,...,...,...
1278437,ad,False,1,Tuesday,23
1327975,ad,False,1,Tuesday,23
1038442,ad,False,3,Tuesday,23


In [65]:
def true_or_false(converted):
    if converted == True:
        return 1
    if converted == False:
        return 0
    else:
        return 'no info'
    
df['converted'] = df['converted'].apply(true_or_false)
df


,test group,converted,total ads,most ads day,most ads hour
user id,,,,,
1069124,ad,0,130,Monday,20
1119715,ad,0,93,Tuesday,22
1144181,ad,0,21,Tuesday,18
1435133,ad,0,355,Tuesday,10
1015700,ad,0,276,Friday,14
...,...,...,...,...,...
1278437,ad,0,1,Tuesday,23
1327975,ad,0,1,Tuesday,23
1038442,ad,0,3,Tuesday,23


In [66]:
def day_of_week(most_ads_day):
    if most_ads_day == 'Monday':
        return 1
    if most_ads_day == 'Tuesday':
        return 2
    if most_ads_day == 'Wednesday':
        return 3
    if most_ads_day == 'Thursday':
        return 4
    if most_ads_day == 'Friday':
        return 5
    if most_ads_day == 'Saturday':
        return 6
    if most_ads_day == 'Sunday':
        return 7
    else:
        return 'no info'
    
df['most ads day'] = df['most ads day'].apply(day_of_week)
df

,test group,converted,total ads,most ads day,most ads hour
user id,,,,,
1069124,ad,0,130,1,20
1119715,ad,0,93,2,22
1144181,ad,0,21,2,18
1435133,ad,0,355,2,10
1015700,ad,0,276,5,14
...,...,...,...,...,...
1278437,ad,0,1,2,23
1327975,ad,0,1,2,23
1038442,ad,0,3,2,23


In [67]:
def time_of_day(most_ads_hour):
    if most_ads_hour >= 5 and most_ads_hour <= 11:
        return 'morning'
    if most_ads_hour >=  12 and most_ads_hour <= 16:
        return 'afternoon'
    if most_ads_hour >= 17 and most_ads_hour < 22:
        return 'evening'
    else:
        return 'night'
    
df['time of day'] = df['most ads hour'].apply(time_of_day)
df['conversion'] = df['converted']/df['total ads']
df

,test group,converted,total ads,most ads day,most ads hour,time of day,conversion
user id,,,,,,,
1069124,ad,0,130,1,20,evening,0.0
1119715,ad,0,93,2,22,night,0.0
1144181,ad,0,21,2,18,evening,0.0
1435133,ad,0,355,2,10,morning,0.0
1015700,ad,0,276,5,14,afternoon,0.0
...,...,...,...,...,...,...,...
1278437,ad,0,1,2,23,night,0.0
1327975,ad,0,1,2,23,night,0.0
1038442,ad,0,3,2,23,night,0.0


##### Рассчитайте вспомогательные показатели для контрольной и тестовой групп

In [68]:
test_group_pivot = (df[['test group', 'total ads', 'converted']]
              .pivot_table(index=['test group'], values = 'converted', aggfunc = ['count','sum'], margins=True))
test_group_pivot.columns = ['sum_of_users', 'sum_of_converted']
test_group_pivot['conversion'] = test_group_pivot['sum_of_converted']/test_group_pivot['sum_of_users']*100
test_group_pivot

,sum_of_users,sum_of_converted,conversion
test group,,,
ad,564577,14423,2.554656
psa,23524,420,1.785411
All,588101,14843,2.523886


Количество посещений сайта (показов объявлений) 14,6 млн раз, суммарное количество совершённых покупок для контрольной группы - 420 раз и у тестовой группы - 14 423 покупки.

Промежуточные данные о показах объявления говорят о конверсии в тестовой группе 2,6%, а в контрольной - 1,8%, что ниже, таким образом показ объявления увеличивает вероятность, что покупатель совершит покупку.

Количество обоих вариантов посадочной страницы посещений сопоставимо, выборки сбалансированные.

In [69]:
test_group_pivot_2 = (df[['test group', 'total ads', 'converted', 'most ads day']]
              .pivot_table(index=['test group', 'most ads day'], values = 'converted', aggfunc = ['count','sum']))
test_group_pivot_2.columns = ['sum_of_users', 'sum_of_converted']
test_group_pivot_2['conversion'] = test_group_pivot_2['sum_of_converted']/test_group_pivot_2['sum_of_users']*100
test_group_pivot_2

sum_of_users  sum_of_converted  conversion
test group most ads day                                            
ad         1                    83571              2778    3.324120
           2                    74572              2270    3.044038
           3                    77418              1963    2.535586
           4                    79077              1711    2.163714
           5                    88805              1995    2.246495
           6                    78802              1679    2.130657
           7                    82332              2027    2.461983
psa        1                     3502                79    2.255854
           2                     2907                42    1.444788
           3                     3490                55    1.575931
           4                     3905                79    2.023047
           5                     3803                62    1.630292
           6                     2858                40    1.399580
           7                     3059                63    2.059497

In [70]:
test_group_pivot_3 = (df[['test group', 'total ads', 'converted', 'most ads day', 'time of day']]
              .pivot_table(index=['test group', 'most ads day', 'time of day'], values = 'converted', aggfunc = ['count','sum']))
test_group_pivot_3.columns = ['sum_of_users', 'sum_of_converted']
test_group_pivot_3['conversion'] = test_group_pivot_3['sum_of_converted']/test_group_pivot_3['sum_of_users']*100
test_group_pivot_3

sum_of_users  sum_of_converted  \
test group most ads day time of day                                   
ad         1            afternoon           31410              1209   
                        evening             23730               763   
                        morning             19483               593   
                        night                8948               213   
           2            afternoon           29209              1065   
                        evening             18298               554   
                        morning             20132               469   
                        night                6933               182   
           3            afternoon           29168               840   
                        evening             18942               510   
                        morning             20914               448   
                        night                8394               165   
           4            afternoon           30312               698   
                        evening             21097               494   
                        morning             18623               338   
                        night                9045               181   
           5            afternoon           32910               794   
                        evening             23857               598   
                        morning             20397               398   
                        night               11641               205   
           6            afternoon           28849               579   
                        evening             22798               597   
                        morning             17489               308   
                        night                9666               195   
           7            afternoon           31462               688   
                        evening             21845               764   
                        morning             20230               351   
                        night                8795               224   
psa        1            afternoon            1439                42   
                        evening               863                18   
                        morning               840                17   
                        night                 360                 2   
           2            afternoon            1201                19   
                        evening               605                12   
                        morning               902                 8   
                        night                 199                 3   
           3            afternoon            1459                27   
                        evening              1046                20   
                        morning               685                 5   
                        night                 300                 3   
           4            afternoon            1553                37   
                        evening               983                19   
                        morning               959                14   
                        night                 410                 9   
           5            afternoon            1571                18   
                        evening               917                27   
                        morning               962                14   
                        night                 353                 3   
           6            afternoon            1144                21   
                        evening               741                11   
                        morning               682                 7   
                        night                 291                 1   
           7            afternoon            1164                28   
                        evening               840                25   
                        morning               720            

In [71]:
sample_1 = df[df['test group'] == 'ad'].groupby(['converted'])["conversion"].mean()
sample_2 = df[df['test group'] == 'psa'].groupby(['converted'])["conversion"].mean()

alpha = .05

results = st.ttest_ind(
    sample_1, 
    sample_2)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Конверсия выборок не равна между собой")
else:
    print("Конверсия выборок равна между собой")


p-значение: 0.8798845396601017
Конверсия выборок равна между собой
